In [14]:
import pandas as pd
import numpy as np
from pprint import pprint

import pickle

In [15]:
with open("D:/cleaned_data.pk", "rb") as f:
    data = pickle.load(f)
    
data.reset_index(drop=True, inplace=True)
print(data.head())
print(data.info())

         DATE  STAR             REVIEW  LIKE
0  2019-12-27     5  부산 지역 화폐 카드 신청 사용     8
1  2019-12-27     5                화이팅     4
2  2019-12-28     5                 게요     4
3  2019-12-28     5   여기저기 사용 부산 경제 도움     4
4  2019-12-28     5           부산 부산 혜택     5
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2871 entries, 0 to 2870
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   DATE    2871 non-null   object
 1   STAR    2871 non-null   int64 
 2   REVIEW  2871 non-null   object
 3   LIKE    2871 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 89.8+ KB
None


# 1. ATM 사용을 위한 데이터 처리

In [16]:
stars = set(data["STAR"])
stars

{1, 2, 3, 4, 5}

In [17]:
authors = data.groupby('STAR')
pprint(authors.groups)
print(type(authors.groups))

{1: Int64Index([  11,   18,   21,   22,   23,   25,   26,   27,   28,   29,
            ...
            2855, 2856, 2857, 2858, 2860, 2861, 2863, 2865, 2868, 2869],
           dtype='int64', length=1207),
 2: Int64Index([  44,   67,   85,  111,  120,  141,  173,  174,  215,  236,
            ...
            2718, 2737, 2739, 2771, 2792, 2799, 2814, 2828, 2862, 2864],
           dtype='int64', length=142),
 3: Int64Index([  14,   24,   31,   32,   33,   37,   38,   41,   43,   46,
            ...
            2716, 2717, 2734, 2744, 2754, 2766, 2830, 2851, 2859, 2866],
           dtype='int64', length=218),
 4: Int64Index([   9,   34,   42,   45,   61,   64,   71,   74,   76,   95,
            ...
            2688, 2728, 2794, 2803, 2824, 2825, 2829, 2840, 2845, 2870],
           dtype='int64', length=214),
 5: Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,   10,
            ...
            2811, 2817, 2819, 2826, 2834, 2835, 2838, 2847, 2850, 2867],
           dtype='

In [18]:
author2doc = {}

for stars, index in authors.groups.items():
    author2doc[stars] = list(index)
    
print(author2doc)

{1: [11, 18, 21, 22, 23, 25, 26, 27, 28, 29, 30, 35, 36, 39, 40, 47, 48, 49, 50, 51, 52, 53, 56, 57, 59, 60, 66, 68, 72, 77, 79, 80, 84, 87, 91, 92, 94, 96, 101, 102, 103, 106, 107, 108, 109, 110, 112, 113, 114, 115, 116, 117, 118, 125, 127, 133, 134, 139, 140, 148, 165, 169, 176, 182, 185, 188, 189, 190, 193, 195, 197, 201, 202, 205, 207, 213, 216, 247, 251, 259, 260, 261, 263, 264, 265, 269, 273, 274, 283, 285, 287, 288, 290, 291, 294, 295, 296, 297, 298, 299, 302, 307, 309, 311, 312, 316, 319, 322, 345, 378, 382, 432, 447, 458, 459, 460, 463, 465, 474, 475, 477, 483, 484, 485, 489, 493, 496, 497, 500, 504, 506, 507, 509, 514, 518, 520, 521, 523, 526, 527, 528, 529, 534, 537, 538, 540, 547, 552, 558, 559, 562, 563, 565, 566, 569, 570, 571, 576, 577, 578, 581, 583, 585, 586, 588, 589, 590, 591, 592, 599, 600, 603, 604, 605, 607, 608, 610, 611, 612, 613, 614, 615, 618, 621, 622, 623, 624, 626, 627, 628, 629, 630, 632, 635, 636, 637, 640, 641, 644, 645, 646, 647, 649, 650, 651, 652, 653

In [19]:
# gensim의 들어갈 데이터 만들기
tokenized_data = [msg.split() for msg in list(data["REVIEW"])]
print(tokenized_data[:10])

[['부산', '지역', '화폐', '카드', '신청', '사용'], ['화이팅'], ['게요'], ['여기저기', '사용', '부산', '경제', '도움'], ['부산', '부산', '혜택'], ['사용', '백전', '혜택'], ['글로벌', '테스트', '베드', '부산'], ['모바일', '지역', '화폐', '설치', '카드', '신청', '구성', '수도', '명절', '카드', '부모', '지역', '페이', '수당', '나중', '복지', '비용', '이걸로', '부모', '기대'], ['출시', '이용'], ['군더더기', '마음', '사업장', '사용', '가능']]


In [20]:
from gensim.models import AuthorTopicModel
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary, bleicorpus
from gensim.matutils import hellinger
from gensim import corpora
from tqdm import tqdm_notebook
from time import time

import os

In [21]:
# ATM에 사용할 Dictionary 만들기
if not os.path.exists('dongnoun_dict'):
    dictionary = corpora.Dictionary(tokenized_data)
    dictionary.save('dongnoun_dict')
    print(dictionary)
else:
    dictionary = Dictionary.load('dongnoun_dict')

# ATM에 사용할 corpus 만들기
if not os.path.exists('dongnoun_corpus'):
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]
    corpora.BleiCorpus.serialize('dongnoun_corpus', corpus)
else:
    corpus = bleicorpus.BleiCorpus('dongnoun_corpus')

Dictionary(1952 unique tokens: ['부산', '사용', '신청', '지역', '카드']...)


In [22]:
# ATM에 들어갈 데이터 확인하기
print("Number of authors : %d" %len(authors))
print("Number of unique tokens : %d" % len(dictionary))
print("Number of documents : %d" % len(corpus))

Number of authors : 5
Number of unique tokens : 1952
Number of documents : 2871


In [23]:
# dictionary함수로 만든 사전에 있는 단어 보기
print(dictionary)
for idx in dictionary:
    print(dictionary[idx])

Dictionary(1952 unique tokens: ['부산', '사용', '신청', '지역', '카드']...)
부산
사용
신청
지역
카드
화폐
화이팅
게요
경제
도움
여기저기
혜택
백전
글로벌
베드
테스트
구성
기대
나중
명절
모바일
복지
부모
비용
설치
수당
수도
이걸로
페이
이용
출시
가능
군더더기
마음
사업장
개인
네트워크
부산시
선택
약정
장사
장애
정보
진행
성장
시민
대표
이름
편리
편의점
결제
기본
코드
가입
다음
사람
이상
회원
적립
충전
한도
마트
상태
상황
하나
확인
건가요
데이터
오류
와이파이
가게
개선
경우
문의
방법
방안
불편
시백
재결
취소
해결
실패
실행
취득
먹통
금액
때문
발급
부탁
시도
입력
자동
하루
한참
가맹점
며칠
정도
조회
감안
계산
동안
보안
지연
처음
검색
문제
서버
안드로이드
애플
언제
접속
제한
삭제
위치
재설
허용
아무것
어딜
감사
관련
동네
만약
만족
설정
얼마
업데이트
유무
전체
캐시
평소
활성
로그인
소용
시스템
신경
안정
장난
포인트
잔액
카페
버그
수정
까지
강화
기간
기능
실적
오늘
요청
메세지
문자
인증
교통
교통카드
내역
후불
관리
지출
합계
이게
초기
개발
실력
쓰레기
에러
인정
전화
최악
고객
다운
답변
센터
실망
자체
보완
걸음
결국
광고
국민
기존
등등
만큼
백만
불가
승인
시간
운영
의견
점심
제기
제때
캐쉬
퇴근
단위
만족도
이용자
재미
직관
총합
한눈
실력자
여분
이번
계좌
등록
뱅크
카카오
뭡니까
슈퍼
오전
일반
지도
공지
이거
주요
택시
띄어쓰기
오탈
조건
표시
내일
문구
화면
겁니까
이것
메시지
서비스
금일
저녁
주니
포스
플로
단계
본인
덕분
오랜만
메뉴
모션
진입
하단
내용
무렵
보유
보이
시안
아래쪽
클릭
필요
현황
활용
실물
아이폰
마지막
번호
비밀
소비
현명
생각
환불
그게
식이
통장
중복
걱정
버튼
만점
삼성
페이도
가정생활
연결
지하철
최고
할인
기념
다운로드
이벤트
정책
동의
필수
비율
바코드
이젠
어제
거지
어디
점검
체크카드
출금
뭔데요
시일
싸이트
건지


In [24]:
# 사람이 이해할 수 있는 형태로 코퍼스 사전 재구성 해보기
[[(dictionary[id],freq) for id, freq in cp] for cp in corpus]
    

[[('부산', 1), ('사용', 1), ('신청', 1), ('지역', 1), ('카드', 1), ('화폐', 1)],
 [('화이팅', 1)],
 [('게요', 1)],
 [('부산', 1), ('사용', 1), ('경제', 1), ('도움', 1), ('여기저기', 1)],
 [('부산', 2), ('혜택', 1)],
 [('사용', 1), ('혜택', 1), ('백전', 1)],
 [('부산', 1), ('글로벌', 1), ('베드', 1), ('테스트', 1)],
 [('신청', 1),
  ('지역', 2),
  ('카드', 2),
  ('화폐', 1),
  ('구성', 1),
  ('기대', 1),
  ('나중', 1),
  ('명절', 1),
  ('모바일', 1),
  ('복지', 1),
  ('부모', 2),
  ('비용', 1),
  ('설치', 1),
  ('수당', 1),
  ('수도', 1),
  ('이걸로', 1),
  ('페이', 1)],
 [('이용', 1), ('출시', 1)],
 [('사용', 1), ('가능', 1), ('군더더기', 1), ('마음', 1), ('사업장', 1)],
 [],
 [('개인', 1),
  ('네트워크', 1),
  ('부산시', 1),
  ('선택', 2),
  ('약정', 1),
  ('장사', 1),
  ('장애', 1),
  ('정보', 1),
  ('진행', 1)],
 [('부산', 1), ('이용', 1), ('성장', 1), ('시민', 1)],
 [('부산', 2),
  ('사용', 1),
  ('혜택', 1),
  ('기대', 1),
  ('이용', 1),
  ('대표', 1),
  ('이름', 2),
  ('편리', 1)],
 [('사용', 1), ('편의점', 1)],
 [('설치', 1)],
 [('사용', 1), ('군더더기', 1)],
 [('사용', 2)],
 [('신청', 1), ('카드', 1), ('결제', 1), ('기본', 1), ('코드', 1)],
 [('가

In [25]:
# Author Topic Model 실행
NUM_TOPICS = 4

if not os.path.exists("dongnoun_model"):
    model = AuthorTopicModel(corpus = corpus, id2word = dictionary.id2token , num_topics = NUM_TOPICS,
                            author2doc = author2doc , passes = 10)
    model.save("dongnoun_model")
else:
    model = AuthorTopicModel.load("dongnoun_model")
    

In [26]:
# 학습 결과 확인하기
model.show_topic(3)

[('사용', 0.06605172565200179),
 ('포인트', 0.03327908941170897),
 ('편리', 0.03249212560756521),
 ('혜택', 0.026950190905236485),
 ('부산', 0.026841084911934515),
 ('적립', 0.024349170215138962),
 ('감사', 0.01609316832104012),
 ('캐시', 0.01564507501269745),
 ('최고', 0.015409181005213329),
 ('지역', 0.015229014943307298)]